# Introduction

학습한 내용을 바탕으로 간단한 커스터마이징을 더한, *사용자 정의 함수* 코드

## scikit-learn 내장 예제 데이터 세트 to DataFrame 

### builtInData_to_dataframe() 함수 

* parameter 
    * dataset : dictionary type 내장 dataset    


* manual 
    1. 내장 dataset 속의 ndarray type 의 data 와 label 을 dataframe 형태로 바꾸어줌.
    2. label 은 dataframe 의 마지막에 추가

In [15]:
import pandas as pd 

def builtInData_to_dataframe(dataset):
    
    dataset_data = dataset.data
    dataset_label = dataset.target
    
    df = pd.DataFrame(data=dataset_data,columns=dataset.feature_names)
    df['label']=dataset_label
    
    return df

In [16]:
### Example 

from sklearn.datasets import load_iris

iris = load_iris()
builtInData_to_dataframe(iris)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## 단순 모델 학습 및 예측 & 평가

### basic_modeling() 함수  

* parameter 
    * df : DataFrame
    * labelName : The name of the column to be used as a label in df (String)
    * testSize : The percentage of test size to separate from the data
    * randomState : Seed
* manual 
    1. dataframe 을 통해 학습 , 테스트 데이터로 분리 후 학습 , 예측 , 평가 진행 
    2. 평가 -  accuracy ( 정확도 ) 사용 
    3. 예측 - DecisionTreeClassifier 알고리즘 사용

In [175]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def basic_modeling(df,labelName,testSize,randomState):
    
    label = df[labelName]
    feature = df.drop(labelName,axis=1)
    
    X_train , X_test , y_train , y_test = train_test_split(feature,label,test_size= testSize,random_state = randomState)
    
    dt_clf = DecisionTreeClassifier()
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    acc = round(accuracy_score(y_test,pred),4)*100

    return acc

In [176]:
### Example 

from sklearn.datasets import load_iris

iris = load_iris()
print('예측 정확도 : ',basic_modeling(builtInData_to_dataframe(iris),'label',0.3, 15),'%')

예측 정확도 :  97.78 %


## 교차 검증 Stratified K-fold
위에서 학습과 테스트 데이터를 분리 한 것처럼 데이터를 분리 할 때 둘 중 한쪽에 과도하게 최적화 되어서 모델이 만들어 지는 것을 '과적합' 이 라고 한다. <br>그로 인해 성능이 저하 될 수 있기 때문에 '교차 검증' 단계를 거쳐야 한다. <br><br>
K-Fold : K 개의 데이터 폴드 세트를 만들어서 K번 만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법<br>
불균형한 분포도를 가진 레이블 데이터 집합 일 경우에 학습 시 레이블 비율을 제대로 반영 할 수 없다. (p.103~106 참고) <br><br>

Stratified K-Fold : 원본 데이터의 레이블 분포를 먼저 고려한 뒤, 위와 동일하게 진행함으로써 문제를 해결해줌. 

### stratified_kfold()

* parameter 
    * df : DataFrame
    * labelName : The name of the column to be used as a label in df (String)
    * n : the number of fold set 
* manual 
    1. X_train , X_test 설정에서 df 의 값만 가져와야 해서 values 사용 
    2. StratifiedKFold 의 split() 를 사용하면, 레이블을 고려하여 알아서 인덱스를 통해 학습과 테스트 데이터가 나뉜다

In [134]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

def stratified_kfold(df,labelName,n):
    
    list = []
    
    dt_clf = DecisionTreeClassifier(random_state=150)
        
    label = df[labelName]
    feature = df.drop(labelName,axis=1)
    
    skf = StratifiedKFold(n_splits=n)
    
    n_iter = 0
    
    for train_index, test_index in skf.split(feature,label):

        X_train , X_test = feature.iloc[train_index].values , feature.iloc[test_index].values
        y_train , y_test = label.iloc[train_index] , label.iloc[test_index] 
        
        dt_clf.fit(X_train,y_train)
        pred = dt_clf.predict(X_test)
        
        n_iter += 1
        
        # 반복 시 마다 정확도 측정 
        accuracy = np.round(accuracy_score(y_test,pred),4)
           
        list.append(accuracy)
        
        if(n_iter == n):
            print(np.round(np.mean(list)*100,2),'%')
        
    return ;

In [138]:
# Example

from sklearn.datasets import load_iris

iris = load_iris()
stratified_kfold(builtInData_to_dataframe(iris),'label',3)

96.67 %


## 교차 검증 cross_val_score
이번에는 sklearn.model_selection 에 실제로 있는 함수를 가져왔다.<br>
앞서 StratifiedKFold  , fit , predict , accuracy_score 해주었던 과정들을 전부 다 내포한 함수로<br>
> cross_val_score(estimator , data , label , scoring = '성능 평가 지표', cv = 폴드 수)

를 이용해서 간편하게 구할 수 있다. <br>
추가로 cross_validate() 을 이용하면 여러 개의 평가 지표를 동시에 사용 할 수 있다. ( p.112 참고 )

## 교차 검증과 하이퍼 파라미터 튜닝을 한 번에 GridSearchCV

GridSearchCV 는 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾게 해준다. 예로 결정 트리 알고리즘의 여러 가지 최적화 파라미터를 순차적으로 적용한다. <br> 
이 때, 테스트 할 하이퍼 파라미터 세트는 딕셔너리 형태로 하이퍼 파라미터의 명칭이 문자열 Key 로 , 값은 리스트형 을 가진다. 

### GSCV()

* parameter 
    * df : DataFrame
    * labelName : The name of the column to be used as a label in df (String)
    * parameters : hyper-parameter in dictionary type 
* manual 
    1. 하이퍼 파라미터와 dataframe , label명 만 넣어주면 DecisionTreeClassifier 알고리즘에 최적의 파라미터를 적용하여 예측 정확도를 측정 해준다. 
    2. 내부적으로는 학습, 테스트 데이터로 분리되고 , 성능 평가 지표를 원한다면 함수 내부에서 변경을 해주어야 한다. <br>
    물론 적용하려는 ML 알고리즘 변경도 함수 수정을 필요로 한다.

In [155]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

def GSCV(df,labelName,parameters,testSize):
    
    label = df[labelName]
    feature = df.drop(labelName,axis=1)
    
    X_train , X_test , y_train , y_test = train_test_split(feature,label,test_size= testSize,random_state = 121)
    
    dtree = DecisionTreeClassifier()
    
    grid_dtree = GridSearchCV(dtree,param_grid=parameters,cv=3,refit=True)
    
    grid_dtree.fit(X_train,y_train)
    
    results = pd.DataFrame(grid_dtree.cv_results_)
    
    print(results[['params','mean_test_score','split0_test_score','split1_test_score','split2_test_score']])
    
    print("\n최적의 파라미터 : ",grid_dtree.best_params_,'\n\n','최고 정확도 :{0:.2f}'.format(grid_dtree.best_score_ * 100),'%')
    
    estimator = grid_dtree.best_estimator_
    
    pred = estimator.predict(X_test)
    
    print('\n테스트 데이터 세트 정확도:{0:.2f}'.format(accuracy_score(y_test,pred)*100),'%')
    
    return ;

In [156]:
## Example

parameters = {'max_depth':[1,2,3],'min_samples_split':[2,3]}

GSCV(builtInData_to_dataframe(iris),'label',parameters,0.2)

                                     params  mean_test_score  \
0  {'max_depth': 1, 'min_samples_split': 2}         0.700000   
1  {'max_depth': 1, 'min_samples_split': 3}         0.700000   
2  {'max_depth': 2, 'min_samples_split': 2}         0.958333   
3  {'max_depth': 2, 'min_samples_split': 3}         0.958333   
4  {'max_depth': 3, 'min_samples_split': 2}         0.975000   
5  {'max_depth': 3, 'min_samples_split': 3}         0.975000   

   split0_test_score  split1_test_score  split2_test_score  
0              0.700                0.7               0.70  
1              0.700                0.7               0.70  
2              0.925                1.0               0.95  
3              0.925                1.0               0.95  
4              0.975                1.0               0.95  
5              0.975                1.0               0.95  

최적의 파라미터 :  {'max_depth': 3, 'min_samples_split': 2} 

 최고 정확도 :97.50 %

테스트 데이터 세트 정확도:96.67 %


## 데이터 전처리 - 레이블 인코딩 , 원-핫 인코딩 

사이킷런의 ML 알고리즘은 문자열 값을 허용하지 않는다. 그래서 모든 문자열 값은 인코딩 돼서 숫자형으로 변환해야 한다. <br>
레이블 인코딩, 원-핫 인코딩이 있는데 전자의 경우 선형 회귀 같은 ML 알고리즘에서 가중치 문제가 생길 수 있기 때문에 여기서는 후자를 쓴다.

### encoding()

* parameter 
    * column : 인코딩 해야 하는 컬럼 
* manual 
    1. LabelEncoder 로 먼저 해당 문자열 칼럼을 숫자로 변환 시켜주고 2차원으로 바꿔준다.<br>
    그 상태에서 OneHotEncoder 를 사용하면 가중치 없이 특정 문자열을 나타낼 때 1로만 표현 된다.

In [160]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import pandas as pd
import numpy as np

def encoding(column):
    # 먼저 숫자값으로 변환 
    encoder = LabelEncoder()
    encoder.fit(items)
    labels = encoder.transform(items)
    
    # 2차원 데이터로 변환 
    labels = labels.reshape(-1,1)
    
    # 원 핫 인코터 적용 
    oh_encoder = OneHotEncoder()
    oh_encoder.fit(labels)
    oh_labels = oh_encoder.transform(labels)
    
    print(oh_labels.toarray())
    
    return ;

In [167]:
## Example

items = ['컴퓨터','시계','컴퓨터','냉장고','침대','시계','시계','시계','컴퓨터','컴퓨터']
encoding(items)

[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


사실 그냥 pandas 의 get_dummies() 쓰면 1줄로 원-핫 인코딩이 된다. 

In [168]:
df = pd.DataFrame({'item':['컴퓨터','시계','컴퓨터','냉장고','침대','시계','시계','시계','컴퓨터','컴퓨터']})

pd.get_dummies(df)

,item_냉장고,item_시계,item_침대,item_컴퓨터
0,0,0,0,1
1,0,1,0,0
2,0,0,0,1
3,1,0,0,0
4,0,0,1,0
5,0,1,0,0
6,0,1,0,0
7,0,1,0,0
8,0,0,0,1
9,0,0,0,1


## 데이터 전처리 - 피쳐 스케일링과 정규화 

피처 스케일링에는 표준화와 정규화가 있다. 서로 다른 피처의 값 범위를 일정한 수준으로 맞추기 위한 작업에서 사용된다.<br>
표준화( StandardScaler ) : 데이터의 피처 각 각이 평균 0 분산 1 인 가우시안 정규 분포를 가진 값으로 변환하는 방법 <br>
정규화( MinMaxScaler ) : 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해주는 개념 <br>
( p.124 식 참고 )<br><br>

사용법은 두 가지 모두 동일하다. <br>
유의해야 할 점은 학습 데이터와 테스트 데이터 모두 동일하게 작업을 해주어야 한다.<br>


### encoding()

* parameter 
    * df : DataFrame

In [192]:
from sklearn.preprocessing import StandardScaler

def scaler(df):
    scaler = StandardScaler()
    
    temp = df['label']
    df = df.drop('label',axis=1)
    
    scaler.fit(df)
    df_scaled = scaler.transform(df)
    # transform 시 ndarray 타입으로 변환된다
    
    dataframe = pd.DataFrame(data=df_scaled,columns=df.columns)
    
    dataframe['label'] = temp
    
    # 사용자 정의 함수 
    print(basic_modeling(dataframe,'label',0.3,300))
    
    return dataframe   

In [193]:
## Example

scaler(builtInData_to_dataframe(iris))

97.78


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,-0.900681,1.019004,-1.340227,-1.315444,0
1,-1.143017,-0.131979,-1.340227,-1.315444,0
2,-1.385353,0.328414,-1.397064,-1.315444,0
3,-1.506521,0.098217,-1.283389,-1.315444,0
4,-1.021849,1.249201,-1.340227,-1.315444,0
...,...,...,...,...,...
145,1.038005,-0.131979,0.819596,1.448832,2
146,0.553333,-1.282963,0.705921,0.922303,2
147,0.795669,-0.131979,0.819596,1.053935,2
148,0.432165,0.788808,0.933271,1.448832,2
